In [ ]:
pip install spacy

In [2]:
import pandas as pd
import string
import unicodedata
import numpy as np
import openpyxl
import spacy
#import transformers
#import torch

In [128]:
df_corpus_uoc = pd.read_excel('data/corpus_con_variaciones_20240427_uoc_ofuscado.xlsx')
numNA = df_corpus_uoc.isna().sum()
print(numNA)
df_corpus_uoc.head()

Unnamed: 0           0
CODIGO_FAQ           0
title_found          0
body texto           2
variaciones          0
num_variaciones      0
motivo             221
intent               0
num_tokens           0
dtype: int64


,Unnamed: 0,CODIGO_FAQ,title_found,body texto,variaciones,num_variaciones,motivo,intent,num_tokens
0,0,433243,Visa Lidl para empleados del grupo Lidl,¿A quién está dirigida? La Visa Lidl Empleado ...,"['Tarjeta para empleados Lidl', 'solicitar UNA...",22,NaN,Visa_Lidl_para_empleados_del_grupo_Lidl,1007
1,1,419915,¿Cómo puedo anular el cobro de una liquidación...,¿Cómo puedo anular el cobro de una liquidación...,"['Anular cobro liquidación tarjeta oficina', '...",20,NaN,_Como_puedo_anular_el_cobro_de_una_liquidacion...,74
2,2,421998,¿Cómo puedo modificar un contrato de tarjeta p...,¿Cómo puedo modificar un contrato de tarjeta p...,"['motivo cambio de límite Tarjeta', 'como renu...",18,NaN,_Como_puedo_modificar_un_contrato_de_tarjeta_p...,122
3,3,423029,¿Cómo modificar la forma o importe de cobro de...,¿Cuál es la comisión por recargar una tarjeta ...,['Qué hago para cambiar la forma de cobro de l...,21,NaN,Como_modificar_la_forma_o_importe_de_cobro_de_...,380
4,4,422618,¿Cuáles son los plazos de recepción de las tar...,¿De que depende el plazo de recepción de las t...,['Por dónde veo los plazos de recepción de las...,29,NaN,Cuales_son_los_plazos_de_recepcion_de_las_tarj...,680


In [129]:
df_corpus_uoc = df_corpus_uoc.dropna(subset=['body texto'])
numNA = df_corpus_uoc.isna().sum()
print(numNA)

Unnamed: 0           0
CODIGO_FAQ           0
title_found          0
body texto           0
variaciones          0
num_variaciones      0
motivo             219
intent               0
num_tokens           0
dtype: int64


In [130]:
df_corpus_uoc.to_pickle('data/corpus_con_variaciones_20240427_uoc_ofuscado.pkl')

In [3]:
df_corpus_uoc = pd.read_pickle('data/corpus_con_variaciones_20240427_uoc_ofuscado.pkl')
df_corpus_uoc.head()

,Unnamed: 0,CODIGO_FAQ,title_found,body texto,variaciones,num_variaciones,motivo,intent,num_tokens
0,0,433243,Visa Lidl para empleados del grupo Lidl,¿A quién está dirigida? La Visa Lidl Empleado ...,"['Tarjeta para empleados Lidl', 'solicitar UNA...",22,NaN,Visa_Lidl_para_empleados_del_grupo_Lidl,1007
1,1,419915,¿Cómo puedo anular el cobro de una liquidación...,¿Cómo puedo anular el cobro de una liquidación...,"['Anular cobro liquidación tarjeta oficina', '...",20,NaN,_Como_puedo_anular_el_cobro_de_una_liquidacion...,74
2,2,421998,¿Cómo puedo modificar un contrato de tarjeta p...,¿Cómo puedo modificar un contrato de tarjeta p...,"['motivo cambio de límite Tarjeta', 'como renu...",18,NaN,_Como_puedo_modificar_un_contrato_de_tarjeta_p...,122
3,3,423029,¿Cómo modificar la forma o importe de cobro de...,¿Cuál es la comisión por recargar una tarjeta ...,['Qué hago para cambiar la forma de cobro de l...,21,NaN,Como_modificar_la_forma_o_importe_de_cobro_de_...,380
4,4,422618,¿Cuáles son los plazos de recepción de las tar...,¿De que depende el plazo de recepción de las t...,['Por dónde veo los plazos de recepción de las...,29,NaN,Cuales_son_los_plazos_de_recepcion_de_las_tarj...,680


In [4]:
df_corpus_uoc2 = df_corpus_uoc[['CODIGO_FAQ','variaciones','motivo']].explode('variaciones').reset_index(drop=True)
df_corpus_uoc2.head()

,CODIGO_FAQ,variaciones,motivo
0,433243,"['Tarjeta para empleados Lidl', 'solicitar UNA...",NaN
1,419915,"['Anular cobro liquidación tarjeta oficina', '...",NaN
2,421998,"['motivo cambio de límite Tarjeta', 'como renu...",NaN
3,423029,['Qué hago para cambiar la forma de cobro de l...,NaN
4,422618,['Por dónde veo los plazos de recepción de las...,NaN


In [5]:
# Defining cosine similarity 
def compute_cosine_similarity(a,b): 
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return (cos_sim)

In [6]:
def preprocess_text(text):
    for p in string.punctuation:
        text = text.replace(p,'')
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore').lower()

In [7]:
df_corpus_uoc2 = df_corpus_uoc2[~df_corpus_uoc2['variaciones'].isna()].reset_index(drop=True)
df_corpus_uoc2['texto'] = df_corpus_uoc2['variaciones'].apply(preprocess_text)

In [8]:
df_corpus_uoc2[df_corpus_uoc2.duplicated(subset='texto',keep=False)].sort_values(by='texto')

,CODIGO_FAQ,variaciones,motivo,texto


In [9]:
df_corpus_uoc2_den = df_corpus_uoc2[~df_corpus_uoc2['motivo'].isna()].drop_duplicates()
df_corpus_uoc2_noden = df_corpus_uoc2[df_corpus_uoc2['motivo'].isna()].drop_duplicates(subset='texto')
df_corpus_uoc_new = pd.concat([df_corpus_uoc2_noden,df_corpus_uoc2_den])

In [10]:
df_corpus_uoc_new = df_corpus_uoc_new.reset_index(drop=True)

In [139]:
df_corpus_uoc_new.to_csv('data/corpus_clean_pre_embed.csv',sep="|", index=None)

In [140]:
df_corpus_uoc_new = pd.read_csv('data/corpus_clean_pre_embed.csv',sep="|")

In [141]:
# all-MiniLM-L6-v2
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")


def mini_embedding(user_query):
    try:
        # Sentences are encoded by calling model.encode()
        embeddings = model.encode(user_query)
    except:
        print ("Error: Embedding error")
    return embeddings

    # Print the embeddings
    '''
    for sentence, embedding in zip(user_query, embeddings):
        print("Sentence:", sentence)
        print("Embedding:", embedding)
        print("")
    '''

In [142]:
df_corpus_uoc_new['embeddings_mini'] = df_corpus_uoc_new['texto'].apply(mini_embedding)

In [143]:
uq = 'que cuota anual tiene una tarjeta de debito'
e = mini_embedding(uq)
df_corpus_uoc_new['score_mini'] = df_corpus_uoc_new['embeddings_mini'].apply(lambda x: compute_cosine_similarity(x, e))

In [144]:
# Load the BERT model. Various models trained on Natural Language Inference (NLI) https://www.sbert.net/docs/pretrained_models.html
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
def bert_embedding(user_query):
    try:
        # Sentences are encoded by calling model.encode()
        embeddings = model.encode(user_query)
    except:
        print ("Error: Embedding error")
    return embeddings


In [145]:
df_corpus_uoc_new['embeddings_bert'] = df_corpus_uoc_new['texto'].apply(bert_embedding)

In [146]:
uq = 'que cuota anual tiene una tarjeta de debito'
e = bert_embedding(uq)
df_corpus_uoc_new['score_bert'] = df_corpus_uoc_new['embeddings_bert'].apply(lambda x: compute_cosine_similarity(x, e))

In [ ]:
pip install es-core-news-sm

In [147]:
# SpaCy's model text embeddings, taking into account efficiency

import spacy
def spacy_embedding(user_query):
    nlp = spacy.load('es_core_news_sm')
    #embeddings = [nlp(sentence).vector for sentence in user_query]
    embeddings = nlp(user_query).vector
    return embeddings

    # Print the embeddings
    '''
    for sentence, embedding in zip(user_query, embeddings):
        print("Sentence:", sentence)
        print("Embedding:", embedding)
        print("")
    '''


In [148]:
#word embeddings using spacy taking into account efficiency
df_corpus_uoc_new['embeddings_spacy'] = df_corpus_uoc_new['texto'].apply(spacy_embedding)

In [149]:
uq = 'que cuota anual tiene una tarjeta de debito'
e = spacy_embedding(uq)
df_corpus_uoc_new['score_spacy'] = df_corpus_uoc_new['embeddings_spacy'].apply(lambda x: compute_cosine_similarity(x, e))

In [ ]:
#pip install -qU torch transformers
'''
import torch
from transformers import AutoModel, AutoTokenizer

# use GPU if available, on mac can use MPS
device = "cuda" if torch.cuda.is_available() else "cpu"

model_id = "intfloat/e5-base-v2"

# initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id).to(device)
model.eval()
'''

In [ ]:
'''
def embeddings_E5(action: str, docs: list[str]):
    
    #action can be 'passage' or 'query
    docs = [f"{action}: {d}" for d in docs]
    # tokenize
    tokens = tokenizer(
        docs, padding=True, max_length=512, truncation=True, return_tensors="pt"
    ).to(device)
    with torch.no_grad():
        # process with model for token-level embeddings
        out = model(**tokens)
        # mask padding tokens
        last_hidden = out.last_hidden_state.masked_fill(
            ~tokens["attention_mask"][..., None].bool(), 0.0
        )
        # create mean pooled embeddings
        doc_embeds = last_hidden.sum(dim=1) / \
            tokens["attention_mask"].sum(dim=1)[..., None]
    return doc_embeds.cpu().numpy().tolist()
    '''

In [ ]:
#word embeddings using E5 model
#df_corpus_uoc_new['embeddings_E5'] = df_corpus_uoc_new['texto'].apply(embeddings_E5, args=('passage',))

In [ ]:
'''
uq = 'que cuota anual tiene una tarjeta de debito'
e = embeddings_E5('query',uq)
df_corpus_uoc_new['score_E5'] = df_corpus_uoc_new['embeddings_E5'].apply(lambda x: compute_cosine_similarity(x, e))
'''

In [150]:
from sentence_transformers import SentenceTransformer

def embeddings_mxbai(docs: list[str]):
    # 1. load model
    model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")
    # 2. Encode
    embeddings = model.encode(docs)
    return embeddings


In [151]:
#word embeddings using mxbai model
df_corpus_uoc_new['embeddings_mxbai'] = df_corpus_uoc_new['texto'].apply(embeddings_mxbai)

In [152]:
uq = 'que cuota anual tiene una tarjeta de debito'
e = embeddings_mxbai(uq)
df_corpus_uoc_new['score_mxbai'] = df_corpus_uoc_new['embeddings_mxbai'].apply(lambda x: compute_cosine_similarity(x, e))

In [ ]:
'''
nlp = spacy.load('es_dep_news_trf')
embeddings = [nlp(sentence).vector for sentence in df_corpus_uoc_new['texto'][0]]
print(embeddings.ner)'''

In [153]:
from sentence_transformers import SentenceTransformer

def embeddings_GIST(docs: list[str]):
    model = SentenceTransformer("avsolatorio/GIST-small-Embedding-v0")
    embeddings = model.encode(docs, convert_to_tensor=False)
    return embeddings

In [155]:
#word embeddings using mxbai model
df_corpus_uoc_new['embeddings_GIST'] = df_corpus_uoc_new['texto'].apply(embeddings_GIST)

In [156]:
uq = 'que cuota anual tiene una tarjeta de debito'
e = embeddings_GIST(uq)
df_corpus_uoc_new['score_GIST'] = df_corpus_uoc_new['embeddings_GIST'].apply(lambda x: compute_cosine_similarity(x, e))

In [11]:
from sentence_transformers import SentenceTransformer

def embeddings_BAAI(docs: list[str]):
    model = SentenceTransformer("BAAI/bge-m3")
    embeddings = model.encode(docs, convert_to_tensor=False)
    return embeddings


In [12]:
#word embeddings using mxbai model
df_corpus_uoc_new['embeddings_BAAI'] = df_corpus_uoc_new['texto'].apply(embeddings_BAAI)

In [13]:
uq = 'que cuota anual tiene una tarjeta de debito'
e = embeddings_BAAI(uq)
df_corpus_uoc_new['score_BAAI'] = df_corpus_uoc_new['embeddings_BAAI'].apply(lambda x: compute_cosine_similarity(x, e))

In [14]:
df_corpus_uoc_new['score_BAAI'].sort_values(ascending=False)

161    0.634031
164    0.606904
59     0.588995
35     0.582008
163    0.565858
         ...   
302    0.266775
231    0.257598
6      0.255942
245    0.255407
145    0.252249
Name: score_BAAI, Length: 439, dtype: float32

In [15]:
# Get index of the 5 highest score_spacy's rows 
indices_max_scores_BAAI = df_corpus_uoc_new['score_BAAI'].nlargest(5).index

# print scores and codes
codes_max_scores_BAAI = df_corpus_uoc_new.loc[indices_max_scores_BAAI, ['CODIGO_FAQ','score_BAAI']] 
print(codes_max_scores_BAAI)

     CODIGO_FAQ  score_BAAI
161      446601    0.634031
164      446600    0.606904
59       435574    0.588995
35       437885    0.582008
163      446603    0.565858


In [16]:
df_corpus_BAAI_final = df_corpus_uoc_new.rename(columns={'embeddings_BAAI':'embeddings'})
df_corpus_BAAI_final.to_csv('../FastAPI/db/data/embedding/variaciones_embedding_BAAI_20240427.csv',sep="|", index=None)
df_corpus_BAAI_final.to_pickle('../FastAPI/db/data/embedding/variaciones_embedding_BAAI_20240427.pkl')

In [157]:
df_corpus_uoc_new['CODIGO_FAQ'].nunique()

439

In [158]:
#df_corpus_uoc_new[['CODIGO_FAQ','embeddings','embeddings_bert', 'embeddings_spacy', 'embeddings_E5','embeddings_mxbai','motivo']].to_csv('../FastAPI/db/data/corpus/variaciones_all_comp_embedding_20240330.csv',sep="|", index=None)
df_corpus_uoc_new[['CODIGO_FAQ','embeddings_mini','embeddings_bert', 'embeddings_spacy', 'embeddings_mxbai','embeddings_GIST','motivo']].to_csv('data/variaciones_all_comp_embedding_uoc_20240427.csv',sep="|", index=None)

In [159]:
df_corpus_uoc_new[['CODIGO_FAQ','embeddings_mini','embeddings_bert', 'embeddings_spacy', 'embeddings_mxbai','embeddings_GIST','motivo']].to_pickle('data/variaciones_all_comp_embedding_uoc_20240427.pkl')

In [ ]:
#df_corpus_uoc_new = pd.read_pickle('../FastAPI/db/data/corpus/variaciones_all_comp_embedding_uoc_20240427.pkl')

In [160]:
df_corpus_uoc_new.head()

,CODIGO_FAQ,variaciones,motivo,texto,embeddings_mini,score_mini,embeddings_bert,score_bert,embeddings_spacy,score_spacy,embeddings_mxbai,score_mxbai,embeddings_GIST,score_GIST
0,433243,"['Tarjeta para empleados Lidl', 'solicitar UNA...",NaN,tarjeta para empleados lidl solicitar una tarj...,"[-0.049297784, 0.06306262, -0.052778855, -0.04...",0.442777,"[-0.049297784, 0.06306262, -0.052778855, -0.04...",0.442777,"[0.06396655, -1.0102829, 0.100738235, 1.127394...",0.572986,"[0.02481748, -0.03403738, -0.14437212, 0.72762...",0.723259,"[-0.030527227, 0.0026669737, 0.01400228, 0.012...",0.819353
1,419915,"['Anular cobro liquidación tarjeta oficina', '...",NaN,anular cobro liquidacion tarjeta oficina proce...,"[-0.026050719, -0.07427596, -0.05052995, -0.07...",0.308574,"[-0.026050719, -0.07427596, -0.05052995, -0.07...",0.308574,"[0.84611166, 0.28720096, 0.834923, -1.2378342,...",0.557424,"[0.48073822, -0.018162876, 0.223723, 0.5368666...",0.719180,"[-0.043238733, -0.026242362, 0.0061765215, -0....",0.842498
2,421998,"['motivo cambio de límite Tarjeta', 'como renu...",NaN,motivo cambio de limite tarjeta como renunciar...,"[-0.12729552, 0.06905968, -0.080994666, -0.075...",0.457227,"[-0.12729552, 0.06905968, -0.080994666, -0.075...",0.457227,"[0.35114053, -0.24000733, 0.6779311, -0.876988...",0.519672,"[-0.028965719, 0.4119485, 0.04523738, 0.877965...",0.738847,"[-0.0527733, 0.0033994697, 0.02993245, -0.0139...",0.814937
3,423029,['Qué hago para cambiar la forma de cobro de l...,NaN,que hago para cambiar la forma de cobro de la ...,"[-0.1008265, 0.026075443, -0.042023044, -0.106...",0.376078,"[-0.1008265, 0.026075443, -0.042023044, -0.106...",0.376078,"[-0.12797742, -0.97483176, 0.7096605, 0.197163...",0.605897,"[0.49374753, -0.26194438, 0.089569956, 0.30814...",0.710258,"[-0.031788237, 0.008151135, 0.04830003, 0.0147...",0.779461
4,422618,['Por dónde veo los plazos de recepción de las...,NaN,por donde veo los plazos de recepcion de las t...,"[-0.06829783, 0.0648448, -0.07408309, -0.08305...",0.435212,"[-0.06829783, 0.0648448, -0.07408309, -0.08305...",0.435212,"[0.054455537, -0.6434433, 0.06104442, 0.659452...",0.503544,"[0.031954594, -0.016352661, -0.09663524, 0.226...",0.696087,"[-0.03212525, -0.019531578, 0.03505223, 0.0109...",0.816509


In [161]:
df_corpus_uoc_new[['CODIGO_FAQ','embeddings_mini','embeddings_bert', 'embeddings_spacy','embeddings_mxbai','embeddings_GIST','motivo','score_mini','score_bert','score_spacy','score_mxbai','score_GIST']].to_csv('data/variaciones_all_results_20240427.csv',sep="|", index=None)

In [ ]:
#df_corpus_uoc_new.sort_values(by='score_E5',ascending=False).all()
#df_corpus_uoc_new.sort_values(by='score_spacy_trf',ascending=False)

In [162]:
#df_corpus_uoc_new.sort_values(by='score_mini',ascending=False)
#df_corpus_uoc_new.sort_values(by='score_bert',ascending=False)
#df_corpus_uoc_new.sort_values(by='score_spacy',ascending=False)
#df_corpus_uoc_new['score_mxbai'].sort_values(ascending=False)
df_corpus_uoc_new['score_GIST'].sort_values(ascending=False)


104    0.894896
122    0.885208
115    0.878429
38     0.877246
131    0.873334
         ...   
297    0.716330
275    0.709927
369    0.708836
235    0.708599
382    0.694944
Name: score_GIST, Length: 439, dtype: float32

In [ ]:
#print("score embeddings_E5: \n",df_corpus_uoc_new['score_E5'].sort_values(ascending=False))

In [163]:
print("score embeddings: \n",df_corpus_uoc_new['score_mini'].sort_values(ascending=False))
print("embeddings_bert: \n",df_corpus_uoc_new['score_bert'].sort_values(ascending=False))
print("score embeddings_spacy: \n",df_corpus_uoc_new['score_spacy'].sort_values(ascending=False))
print("score embeddings_mxbai: \n",df_corpus_uoc_new['score_mxbai'].sort_values(ascending=False))
print("score embeddings_GIST: \n",df_corpus_uoc_new['score_GIST'].sort_values(ascending=False))

score embeddings: 
 59     0.655298
104    0.644096
118    0.637363
73     0.634919
98     0.633164
         ...   
378    0.194065
438    0.187185
225    0.185076
316    0.176290
366    0.169287
Name: score_mini, Length: 439, dtype: float32
embeddings_bert: 
 59     0.655298
104    0.644096
118    0.637363
73     0.634919
98     0.633164
         ...   
378    0.194065
438    0.187185
225    0.185076
316    0.176290
366    0.169287
Name: score_bert, Length: 439, dtype: float32
score embeddings_spacy: 
 199    0.743910
42     0.719312
164    0.712431
35     0.699497
99     0.699264
         ...   
409    0.295848
354    0.282002
437    0.237751
156    0.237059
43     0.208197
Name: score_spacy, Length: 439, dtype: float32
score embeddings_mxbai: 
 164    0.845103
59     0.827778
35     0.816434
104    0.802041
180    0.800790
         ...   
259    0.548262
223    0.543880
356    0.541532
274    0.537967
278    0.532905
Name: score_mxbai, Length: 439, dtype: float32
score embeddings_GI

In [164]:
# Get index of the 5 highest score_mini's rows 
indices_max_scores_mini = df_corpus_uoc_new['score_mini'].nlargest(5).index

# print scores and codes
codes_max_scores_mini = df_corpus_uoc_new.loc[indices_max_scores_mini, ['CODIGO_FAQ','score_mini']] 
print(codes_max_scores_mini)

     CODIGO_FAQ  score_mini
59       435574    0.655298
104      418183    0.644096
118  1986776089    0.637363
73       418649    0.634919
98       308737    0.633164


In [165]:
# Get index of the 5 highest score_bert's rows 
indices_max_scores_bert = df_corpus_uoc_new['score_bert'].nlargest(5).index

# print scores and codes
codes_max_scores_bert = df_corpus_uoc_new.loc[indices_max_scores_bert, ['CODIGO_FAQ','score_bert']] 
print(codes_max_scores_bert)

     CODIGO_FAQ  score_bert
59       435574    0.655298
104      418183    0.644096
118  1986776089    0.637363
73       418649    0.634919
98       308737    0.633164


In [166]:
# Get index of the 5 highest score_spacy's rows 
indices_max_scores_spacy = df_corpus_uoc_new['score_spacy'].nlargest(5).index

# print scores and codes
codes_max_scores_spacy = df_corpus_uoc_new.loc[indices_max_scores_spacy, ['CODIGO_FAQ','score_spacy']] 
print(codes_max_scores_spacy)

     CODIGO_FAQ  score_spacy
199      418363     0.743910
42       431150     0.719312
164      446600     0.712431
35       437885     0.699497
99       192252     0.699264


In [167]:
# Get index of the 5 highest score_mxbai's rows 
indices_max_scores_mxbai = df_corpus_uoc_new['score_mxbai'].nlargest(5).index

# print scores and codes
codes_max_scores_mxbai = df_corpus_uoc_new.loc[indices_max_scores_mxbai, ['CODIGO_FAQ','score_mxbai']] 
print(codes_max_scores_mxbai)

     CODIGO_FAQ  score_mxbai
164      446600     0.845103
59       435574     0.827778
35       437885     0.816434
104      418183     0.802041
180      419303     0.800790


In [168]:
# Get index of the 5 highest score_GIST's rows 
indices_max_scores_GIST= df_corpus_uoc_new['score_GIST'].nlargest(5).index

# print scores and codes
codes_max_scores_GIST = df_corpus_uoc_new.loc[indices_max_scores_GIST, ['CODIGO_FAQ','score_GIST']] 
print(codes_max_scores_GIST)

     CODIGO_FAQ  score_GIST
104      418183    0.894896
122      418606    0.885208
115      422985    0.878429
38       442151    0.877246
131      446104    0.873334


In [169]:
df_corpus_final = df_corpus_uoc_new[['CODIGO_FAQ','embeddings_GIST','motivo']]
df_corpus_final = df_corpus_final.rename(columns={'embeddings_GIST':'embeddings'})
df_corpus_final.to_csv('../FastAPI/db/data/embedding/variaciones_embedding_gist_20240427.csv',sep="|", index=None)
df_corpus_final.to_pickle('../FastAPI/db/data/embedding/variaciones_embedding_gist_20240427.pkl')

In [170]:
df_corpus_final_mini = df_corpus_uoc_new[['CODIGO_FAQ','embeddings_mini','motivo']]
df_corpus_final_mini = df_corpus_final_mini.rename(columns={'embeddings_mini':'embeddings'})
df_corpus_final_mini.to_csv('../FastAPI/db/data/embedding/variaciones_embedding_mini_20240427.csv',sep="|", index=None)
df_corpus_final_mini.to_pickle('../FastAPI/db/data/embedding/variaciones_embedding_mini_20240427.pkl')

In [ ]:
#df_load = pd.read_pickle('../FastAPI/db/data/corpus/variaciones_embedding_gist_20240407.pkl')
#df_load = pd.read_csv('../FastAPI/db/data/corpus/variaciones_embedding_gist_20240407_str.csv',sep="|")
#df_load.head()
#df_corpus_final.to_csv('../FastAPI/db/data/corpus/variaciones_embedding_gist_20240407.csv',sep="|", index=None)
#df_corpus_final.to_pickle('../FastAPI/db/data/corpus/variaciones_embedding_gist_20240407.pkl')

In [ ]:
def to_lista(text):
    if str(type(text)) == "<class 'str'>":
        return text.replace("'","")[1:-1].split('\n ')
    else:
        return text

In [ ]:
#df_corpus['variaciones'] = df_corpus['variaciones'].apply(to_lista)

In [171]:
df_corpus_proj = pd.read_pickle('data/corpus_con_variaciones_20240427_uoc_ofuscado.pkl')
df_corpus_proj.head()
df_corpus_proj[['CODIGO_FAQ','title_found','body texto','variaciones','num_variaciones','motivo','intent','num_tokens']].to_pickle('../FastAPI/db/data/corpus/corpus_variaciones_20240427.pkl')
df_corpus_proj[['CODIGO_FAQ','title_found','body texto','variaciones','num_variaciones','motivo','intent','num_tokens']].to_csv('../FastAPI/db/data/corpus/corpus_variaciones_20240427.csv',sep="|", index=None)